In [111]:
from nltk import bigrams, trigrams, ngrams
from collections import Counter, defaultdict
import random

from nltk import tokenize
file = open("all_unis_reddit_data.txt","r+")

p = file.read()
tokenized = tokenize.sent_tokenize(p.lower())
word_tokenized_init = []
for sent in tokenized:
    word_tokenized_init.append(tokenize.word_tokenize(sent))

word_tokenized = []
for sent in word_tokenized_init:
    for i in range(len(sent)):
        boolean = random.choice([True, False])
        if boolean:
            sent[i] = "UNK"
    word_tokenized.append(sent)

# Create a placeholder for model
bigram_model = defaultdict(lambda: defaultdict(lambda: 0))

# Count frequency of co-occurance  
for sentence in word_tokenized:
    for w1, w2 in bigrams(sentence, pad_right=True, pad_left=True):
        bigram_model[w1][w2] += 1
# Let's transform the counts to probabilities
for w1 in bigram_model:
    total_count = float(sum(bigram_model[w1].values()))
    for w2 in bigram_model[w1]:
        bigram_model[w1][w2] = (bigram_model[w1][w2] + 1)/(total_count+48167)

In [117]:
print(word_tokenized[4])

['fuck', 'UNK', 'UNK', 'UNK', 'UNK', 'proud', 'UNK', 'be', 'a', 'cmu', 'student', '!']


In [118]:
bigram_model["the"]["bathroom"]

8.788780493929464e-05

In [119]:
trigram_model = defaultdict(lambda: defaultdict(lambda: 0))

# Count frequency of co-occurance  
for sentence in word_tokenized:
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        trigram_model[(w1, w2)][w3] += 1
 # Let's transform the counts to probabilities
for w1_w2 in trigram_model:
    total_count = float(sum(trigram_model[w1_w2].values()))
    for w3 in trigram_model[w1_w2]:
        trigram_model[w1_w2][w3] = (trigram_model[w1_w2][w3] + 1)/(total_count+48167)

In [120]:
trigram_model["on", "campus"]["at"]

4.132743728561392e-05

In [121]:
fourgram_model = defaultdict(lambda: defaultdict(lambda: 0))

n = 4

# Count frequency of co-occurance  
for sentence in word_tokenized:
    ngrams = [sentence[i:i+n] for i in range(len(sentence)-n+1)]
    for w1, w2, w3, w4 in ngrams:
        fourgram_model[(w1, w2, w3)][w4] += 1
 # Let's transform the counts to probabilities
for w1_w2_w3 in fourgram_model:
    total_count = float(sum(fourgram_model[w1_w2_w3].values()))
    for w4 in fourgram_model[w1_w2_w3]:
        fourgram_model[w1_w2_w3][w4] = ((fourgram_model[w1_w2_w3][w4] + 1)/(total_count+48167))

In [122]:
fourgram_model["on", "campus", "at"]

defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
            {'east': 4.1521341969772464e-05})

In [142]:
import pandas as pd
data = pd.read_csv("governer_and_preceding_words_extracted.csv")
data['preceding_prob']=0
data.head()

,Unnamed: 0,university,sent_id,original_sentence,translation,word_id,first_cs_word_form,first_cs_word_translation,governer_first_cs_word,preceding_word,governer_trans,preceding_trans,Unnamed: 12,Unnamed: 13,preceding_prob
0,0,CMU,2,rich neighborhood ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 ...,富人 区 ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 就 是 公园 ， 附近 还...,1,rich,富人,neighborhood,NaN,社区,NaN,NaN,NaN,0
1,1,CMU,4,lease 从 八月 开始 ， 具体 日期 可以 商量 。,租约 从 八月 开始 ， 具体 日期 可以 商量 。,1,lease,租约,商量,NaN,To discuss,NaN,NaN,NaN,0
2,2,CMU,5,house 一共 有 两 层 + 地下室 。,房子 一共 有 两 层 + 地下室 。,1,house,房子,有,NaN,There are,NaN,NaN,NaN,0
3,3,CMU,6,一共 有 两 个 可以 洗澡 的 bathroom 。,一共 有 两 个 可以 洗澡 的 浴室 。,8,bathroom,浴室,有,的,There are,The,NaN,NaN,0
4,4,CMU,9,整个 house 家具 齐全 。,整个 房子 家具 齐全 。,2,house,房子,家具,整个,furniture,the whole,NaN,NaN,0


In [143]:
str(data.at[1, 'preceding_trans']).split()[0]

'nan'

In [144]:
for index, row in data.iterrows():
    cell = str(row["preceding_trans"]).split()
    if len(cell) == 1 and str(cell[0]) != 'nan':
        w1 = str(cell[0]).lower()
        w2 = str(row["first_cs_word_form"]).lower()
        prob = bigram_model[w1][w2]
        if prob == 0:
            prob = bigram_model["UNK"][w2]
        if prob == 0:
            prob = bigram_model[w1]["UNK"]
        if prob == 0:
            prob = bigram_model["UNK"]["UNK"]
        data.loc[index, 'preceding_prob'] = prob
        #row['preceding probability'] = bigram_model[w1][w2]
    elif len(cell) == 2:
        w1 = str(cell[0]).lower()
        w2 = str(cell[1]).lower()
        w3 = str(row["first_cs_word_form"]).lower()
        prob = trigram_model[w1, w2][w3]
        if prob == 0:
            prob = trigram_model["UNK", w2][w3]
        if prob == 0:
            prob = trigram_model[w1, "UNK"][w3]
        if prob == 0:
            prob = trigram_model[w1, w2]["UNK"]
        if prob == 0:
            prob = trigram_model["UNK", w2]["UNK"]
        if prob == 0:
            prob = trigram_model[w1, "UNK"]["UNK"]
        if prob == 0:
            prob = trigram_model["UNK", "UNK"][w3]
        if prob == 0:
            prob = trigram_model["UNK", "UNK"]["UNK"]
        data.loc[index, 'preceding_prob'] = prob
        #row['preceding probability'] = trigram_model[w1, w2][w3]
    elif len(str(cell).split()) == 3:
        w1 = str(cell[0]).lower()
        w2 = str(cell[1]).lower()
        w3 = str(cell[2]).lower()
        w4 = str(row["first_cs_word_form"]).lower()
        prob = fourgram_model[w1, w2, w3][w4]
        if prob == 0:
            prob = fourgram_model["UNK", w2, w3][w4]
        if prob == 0:
            prob = fourgram_model[w1, "UNK", w3][w4]
        if prob == 0:
            prob = fourgram_model[w1, w2, "UNK"][w4]
        if prob == 0:
            prob = fourgram_model[w1, w2, w3]["UNK"]
        if prob == 0:
            prob = fourgram_model["UNK", "UNK", w3][w4]
        if prob == 0:
            prob = fourgram_model["UNK", w2, "UNK"][w4]
        if prob == 0:
            prob = fourgram_model["UNK", w2, w3]["UNK"]
        if prob == 0:
            prob = fourgram_model[w1, "UNK", "UNK"][w4]
        if prob == 0:
            prob = fourgram_model[w1, "UNK", w3]["UNK"]
        if prob == 0:
            prob = fourgram_model[w1, w2, "UNK"]["UNK"]
        if prob == 0:
            prob = fourgram_model["UNK", "UNK", "UNK"][w4]
        if prob == 0:
            prob = fourgram_model[w1, "UNK", "UNK"]["UNK"]
        if prob == 0:
            prob = fourgram_model["UNK", w2, "UNK"]["UNK"]
        if prob == 0:
            prob = fourgram_model["UNK", "UNK", w3]["UNK"]
        if prob == 0:
            prob = fourgram_model["UNK", "UNK", "UNK"]["UNK"]
        data.loc[index, 'preceding_prob'] = prob
        #row['preceding probability'] = fourgram_model[w1, w2, w3][w4]
        

In [145]:
data.head(20)

,Unnamed: 0,university,sent_id,original_sentence,translation,word_id,first_cs_word_form,first_cs_word_translation,governer_first_cs_word,preceding_word,governer_trans,preceding_trans,Unnamed: 12,Unnamed: 13,preceding_prob
0,0,CMU,2,rich neighborhood ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 ...,富人 区 ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 就 是 公园 ， 附近 还...,1,rich,富人,neighborhood,NaN,社区,NaN,NaN,NaN,0.000000
1,1,CMU,4,lease 从 八月 开始 ， 具体 日期 可以 商量 。,租约 从 八月 开始 ， 具体 日期 可以 商量 。,1,lease,租约,商量,NaN,To discuss,NaN,NaN,NaN,0.000000
2,2,CMU,5,house 一共 有 两 层 + 地下室 。,房子 一共 有 两 层 + 地下室 。,1,house,房子,有,NaN,There are,NaN,NaN,NaN,0.000000
3,3,CMU,6,一共 有 两 个 可以 洗澡 的 bathroom 。,一共 有 两 个 可以 洗澡 的 浴室 。,8,bathroom,浴室,有,的,There are,The,NaN,NaN,0.000088
4,4,CMU,9,整个 house 家具 齐全 。,整个 房子 家具 齐全 。,2,house,房子,家具,整个,furniture,the whole,NaN,NaN,0.000078
5,5,CMU,10,后院 ： deck 。,后院 ： 阳台 。,3,deck,阳台,阳台,：,The balcony,:,NaN,NaN,0.000006
6,6,CMU,13,空调 有 window ac 和 暖气 。,空调 有 窗口 式 冷气机 和 暖气 。,3,window,窗口,ac,有,空调,There are,NaN,NaN,0.003321
7,7,CMU,14,入住 时间 flexible ， 可 长期 入住 。,入住 时间 灵活 ， 可 长期 入住 。,3,flexible,灵活,入住,时间,to stay in,time,NaN,NaN,0.000011
8,8,CMU,15,感兴趣 的 同学 ， 欢迎 email （ 请 注明 “ 租房 ” ） ， 请 简单 介绍 ...,感兴趣 的 同学 ， 欢迎 发送 邮件 （ 请 注明 “ 租房 ” ） ， 请 简单 介绍 ...,6,email,发送,欢迎,欢迎,welcome,welcome,NaN,NaN,0.000165
9,9,CMU,2,rich neighborhood ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 ...,富人 区 ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 就 是 公园 ， 附近 还...,1,rich,富人,neighborhood,NaN,社区,NaN,NaN,NaN,0.000000


In [147]:
data.to_csv("with_ngram_probs.csv", index = False)

In [149]:
!brew install cmake boost eigen

Error: 
  homebrew-core is a shallow clone.
To `brew update`, first run:
  git -C /usr/local/Homebrew/Library/Taps/homebrew/homebrew-core fetch --unshallow
This command may take a few minutes to run due to the large size of the repository.
This restriction has been made on GitHub's request because updating shallow
clones is an extremely expensive operation due to the tree layout and traffic of
Homebrew/homebrew-core and Homebrew/homebrew-cask. We don't do this for you
automatically to avoid repeatedly performing an expensive unshallow operation in
CI systems (which should instead be fixed to not use shallow clones). Sorry for
the inconvenience!
==> Downloading https://homebrew.bintray.com/bottles/cmake-3.19.6.big_sur.bottle
==> Downloading from https://d29vzk4ow07wi7.cloudfront.net/2b57a88a40946e2340fde
######################################################################## 100.0%
==> Pouring cmake-3.19.6.big_sur.bottle.tar.gz
==> Caveats
Emacs Lisp files have been installed to:
  /us

==> Pouring readline-8.1.big_sur.bottle.tar.gz
==> Caveats
readline is keg-only, which means it was not symlinked into /usr/local,
because macOS provides BSD libedit.

For compilers to find readline you may need to set:
  export LDFLAGS="-L/usr/local/opt/readline/lib"
  export CPPFLAGS="-I/usr/local/opt/readline/include"

==> Summary
🍺  /usr/local/Cellar/readline/8.1: 48 files, 1.6MB
==> Installing postgresql
==> Pouring postgresql-13.2_1.big_sur.bottle.tar.gz
==> Caveats
To migrate existing data from a previous major version of PostgreSQL run:
  brew postgresql-upgrade-database

This formula has created a default database cluster with:
  initdb --locale=C -E UTF-8 /usr/local/var/postgres
For more details, read:
  https://www.postgresql.org/docs/13/app-initdb.html

To have launchd start postgresql now and restart at login:
  brew services start postgresql
Or, if you don't want/need a background service you can just run:
  pg_ctl -D /usr/local/var/postgres start
==> Summary
🍺  /usr/loca

In [151]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz |tar xz
!mkdir kenlm/build
!cd kenlm/build
!cmake ..
!make -j2

--2021-04-12 19:31:08--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491090 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 479.58K  1.50MB/s    in 0.3s    

2021-04-12 19:31:08 (1.50 MB/s) - written to stdout [491090/491090]

CMake Error: The source directory "/Users/Debasmita/Desktop" does not appear to contain CMakeLists.txt.
Specify --help for usage, or press the help button on the CMake GUI.
make: *** No targets specified and no makefile found.  Stop.
